In [0]:
!pip install gym

In [0]:
import gym
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque

# Create the Cart-Pole game environment
env = gym.make('CartPole-v0')


class QNetwork:
    def __init__(self, learning_rate=0.01, state_size=4,
                 action_size=2, hidden_size=10):
        # state inputs to the Q-network
        self.model = Sequential()

        self.model.add(Dense(hidden_size, activation='relu',
                             input_dim=state_size))
        self.model.add(Dense(hidden_size, activation='relu'))
        self.model.add(Dense(action_size, activation='linear'))

        self.model.compile(loss='mse', optimizer=Adam(lr=learning_rate))


class Memory():
    def __init__(self, max_size=1000):
        #dequeはmaxlenを記憶する箱で、maxlenを超えると古いものから押し出されていく
        self.buffer = deque(maxlen=max_size)

    def add(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)),
                               size=batch_size,
                               replace=False)
        return [self.buffer[ii] for ii in idx]


train_episodes = 20#1000          # max number of episodes to learn from
max_steps = 200#200                # max steps in an episode
gamma = 0.99                   # future reward discount

# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
hidden_size = 16               # number of units in each Q-network hidden layer
learning_rate = 0.001         # Q-network learning rate

# Memory parameters
memory_size = 10000            # memory capacity
batch_size = 32                # experience mini-batch size
pretrain_length = batch_size   # number experiences to pretrain the memory

mainQN = QNetwork(hidden_size=hidden_size, learning_rate=learning_rate)

In [4]:
mainQN.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 386
Trainable params: 386
Non-trainable params: 0
_________________________________________________________________


In [0]:
###################################
## Populate the experience memory
###################################

# Initialize the simulation
#現時点での状態を表す為の初期化
#初期化のたびに異なるstateを作っている
env.reset()
#env.action_space.sample()は01のどちらかをランダムに出す
state, reward, done, _ = env.step(env.action_space.sample())
#現状パラメーターstateを(1,4)のnumpy shapeにする
state = np.reshape(state, [1, 4])
#メモリーの初期化
memory = Memory(max_size=memory_size)


#env.step(action) で (array([-0.04418486,  0.34338878,  0.01652308, -0.52337281]), 1.0, False, {})　が返る

In [0]:
#############ここのループはおそらく32個(pretrain_length)の強化学習テーブルをつくる為のもの?##############
for ii in range(pretrain_length):
    # ヴィジュアル化させないためのuncomment
    # env.render()

    #ランダムに0か1（右か左）を返す
    action = env.action_space.sample()
    #01(右左)の選択結果としての現状パラメーター、報酬、終了かどうかを返す
    #なぜ変数名がnext_stateになっているかというと、上段セルPopulate the experience memoryですでにstateができているので、その次を用意する為
    next_state, reward, done, _ = env.step(action)
    #現状パラメーターを(1,4)のnumpy shapeにする
    next_state = np.reshape(next_state, [1, 4])

    if done:
        #失敗に終わったので次につながるパラメーター(1,4)を0で埋める
        next_state = np.zeros(state.shape)
        #今回の(現状パラメーター、右左、報酬-1、次につながるパラメーター)を記録
        #言いたいことは、この状態(state)で右左(01)のactionをしたら失敗しました。だから報酬は-1でnext_stateは0です
        memory.add((state, action, reward, next_state))
        #現在の状態をリセット
        env.reset()
        #新しい状態に初期化
        state, reward, done, _ = env.step(env.action_space.sample())
        #次のための現状パラメーターを用意
        state = np.reshape(state, [1, 4])
    else:
        #今回の(現状パラメーター、右左、報酬1、次につながるパラメーター)を記録
        #言いたいことは、この状態(state)で右左(01)のactionをしたら成功しました。だから報酬は1でnext_stateは繰越です
        memory.add((state, action, reward, next_state))
        #次のための現状パラメーターを用意
        state = next_state

In [7]:
#############
## Training
#############
#ここのループで強化学習
#最大20ループ
step = 0
for ep in range(1, train_episodes):
    total_reward = 0
    #tはステップ数をカウントする為
    t = 0
    #1ループにつき最大200ステップ
    while t < max_steps:
        step += 1
        # Uncomment this next line to watch the training
        #env.render()
        
        ##################### 01のactionを決めるためのcode ####################
        #explore_pは1未満の数値になる。ループ回数ステップ回数増加とともにstep数値が大きくなるとexplore_pは0に近づいていく
        #つまりステップ回数が増えていくとランダム数値を選ぶ確率が低くなっていく
        explore_p = explore_stop + (explore_start - explore_stop)*np.exp(-decay_rate*step)
        #np.random.rand()は0から1までのランダム数値(float)を返す
        #下記if statementは過学習防止のため、ランダム数値か学習modelのpredictかを振り分けてaction(01)を決めている
        if explore_p > np.random.rand():
            # ランダムでaction決定
            action = env.action_space.sample()
        else:
            # Q-networkによるaction決定
            Qs = mainQN.model.predict(state)[0]
            action = np.argmax(Qs)
        ##################### 01のactionを決めるためのcode end ####################
            
                                    #---#
            
        ##################### actionによる結果生成のcode ####################
        #新しい状況パラメーターや報酬などを生成
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, 4])
        total_reward += reward
        ##################### actionによる結果生成のcode end ####################

                                    #---#
        
        ##################### 結果合否からの条件分岐によるstate生成code ####################
        if done:
            #失敗に終わったので次につながるパラメーターを0で埋める
            next_state = np.zeros(state.shape)
            #今回のステップを終了(while構文)させるため、上限の200をセット
            t = max_steps

            print('Episode: {}'.format(ep),
                  'Total reward: {}'.format(total_reward),
                  'Explore P: {:.4f}'.format(explore_p))

            #結果を記録
            memory.add((state, action, reward, next_state))
            #現在の状態をリセット
            env.reset()
            #新しい状態に初期化
            state, reward, done, _ = env.step(env.action_space.sample())
            state = np.reshape(state, [1, 4])
        else:
            #今回の(現状パラメーター、右左、報酬1、次につながるパラメーター)を記録
            memory.add((state, action, reward, next_state))
            #次のための現状パラメーターを用意
            state = next_state
            #ステップ回数を更新
            t += 1
        ##################### 結果合否からの条件分岐によるstate生成code end ####################

                                    #---#
        
        ##################### modelの学習code ####################
        # Replay
        #説明変数の箱を用意
        inputs = np.zeros((batch_size, 4))
        #目的変数の箱を用意
        targets = np.zeros((batch_size, 2))
        #memory.sample(batch_size)で強化学習テーブルの呼び出し
        minibatch = memory.sample(batch_size)
        #このループで強化学習テーブルを更新
        for i, (state_b, action_b, reward_b, next_state_b) in enumerate(minibatch):
            #強化学習テーブルから説明変数を取り出してi番目に入れる
            inputs[i:i+1] = state_b
            target = reward_b
            #.all(axis=1)は全てが0以上(True)かを判定する。他に .any()もある
            #よって、下記の条件式はnext_state_bが全て0ではない、成功で終わった学習セットである意味
            if not (next_state_b == np.zeros(state_b.shape)).all(axis=1):
                #このpredictで出てくる数字は0から1の確率数値 exmple[[0.4325,0.1213]] 
                #target_Q = mainQN.model.predict(next_state_b)[0]
                #np.argmax()は最大値のindexを返すが、np.amax()は最大値そのものを返す
                target = reward_b + gamma * np.amax(mainQN.model.predict(next_state_b)[0])
            targets[i] = mainQN.model.predict(state_b)
            targets[i][action_b] = target
            
        #このステップでの結果をmodelにfit(学習)
        #ここでの目的変数(targets)は、(32,2)のシェイプで、右左(01)の時の数値が格納されている
        #よって、predictされた際には2つの数値が返され、argmaxで大きい方を呼び出す。
        mainQN.model.fit(inputs, targets, epochs=10, verbose=0)
        ##################### modelの学習code end ####################

Episode: 1 Total reward: 3.0 Explore P: 0.9997
Episode: 2 Total reward: 17.0 Explore P: 0.9980
Episode: 3 Total reward: 23.0 Explore P: 0.9958
Episode: 4 Total reward: 15.0 Explore P: 0.9943
Episode: 5 Total reward: 20.0 Explore P: 0.9923
Episode: 6 Total reward: 12.0 Explore P: 0.9911
Episode: 7 Total reward: 16.0 Explore P: 0.9896
Episode: 8 Total reward: 15.0 Explore P: 0.9881
Episode: 9 Total reward: 21.0 Explore P: 0.9860
Episode: 10 Total reward: 10.0 Explore P: 0.9851
Episode: 11 Total reward: 14.0 Explore P: 0.9837
Episode: 12 Total reward: 17.0 Explore P: 0.9820
Episode: 13 Total reward: 27.0 Explore P: 0.9794
Episode: 14 Total reward: 14.0 Explore P: 0.9781
Episode: 15 Total reward: 23.0 Explore P: 0.9758
Episode: 16 Total reward: 41.0 Explore P: 0.9719
Episode: 17 Total reward: 19.0 Explore P: 0.9701
Episode: 18 Total reward: 20.0 Explore P: 0.9682
Episode: 19 Total reward: 24.0 Explore P: 0.9659
